# Question 1 : Explication de type (1-1)

## Contexte du problème

Dans le concours de l'internat de médecine, les candidats sont classés selon une somme pondérée de leurs notes. L'objectif est de fournir une **explication compréhensible** de pourquoi un candidat x est mieux classé qu'un candidat y.

Au lieu de simplement montrer la formule de somme pondérée, on souhaite décomposer l'explication en **trade-offs** plus simples.

### Définitions

- **contribution[i]** = weight[i] × (x[i] - y[i])
- **pros(x,y)** = critères où x est meilleur que y (contribution > 0)
- **cons(x,y)** = critères où y est meilleur que x (contribution < 0)
- **Trade-off (1-1)** : paire (P, C) où P ∈ pros, C ∈ cons, et contribution[P] + contribution[C] > 0
- **Explication (1-1)** : ensemble de trade-offs disjoints couvrant tous les critères de cons(x,y)

## Formulation mathématique

### Variables de décision
- **z[p,c]** ∈ {0,1} pour chaque paire (p ∈ pros, c ∈ cons) avec contribution[p] + contribution[c] > 0
  - z[p,c] = 1 si on utilise le trade-off (p,c) dans l'explication

### Contraintes

**C1 : Couverture complète de cons**
$$\forall c \in \text{cons}(x,y) : \sum_{p \in \text{pros}} z_{p,c} = 1$$

**C2 : Usage unique des éléments de pros**
$$\forall p \in \text{pros}(x,y) : \sum_{c \in \text{cons}} z_{p,c} \leq 1$$

**C3 : Validité des trade-offs**
- Variables créées uniquement pour les paires où contribution[p] + contribution[c] ≥ ε

### Fonction objectif
$$\min \sum_{p,c} z_{p,c}$$

Minimiser le nombre de trade-offs (longueur de l'explication).

In [18]:
# Imports
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

## Fonctions utilitaires


Calcule les contributions de chaque critère a la somme pondérée dans x > y
Contributions=(weight[i] * (x[i] - y[i]))


In [11]:
def compute_contributions(x_scores, y_scores, weights):

    return weights * (x_scores - y_scores)



Classifie les criteres en pros, cons et neutral

pros: indices des criteres favorables a x
cons: indices des criteres favorables a y
neutral: indices des criteres neutres

In [12]:
def classify_criteria(contributions, epsilon=1e-6):

    pros = [i for i, c in enumerate(contributions) if c > epsilon]
    cons = [i for i, c in enumerate(contributions) if c < -epsilon]
    neutral = [i for i, c in enumerate(contributions) if abs(c) <= epsilon]
    
    return pros, cons, neutral

In [13]:
def tradeoffs_1_1_table(contributions, pros, cons, criteria_names):
    """
    Enumerate ALL candidate pairs (P,C) with P in pros and C in cons,
    and mark which ones are valid (Delta[P] + Delta[C] > 0).
    Returns a DataFrame for clean display in the notebook.
    """
    rows = []
    for p in pros:
        for c in cons:
            s = float(contributions[p] + contributions[c])
            rows.append({
                "P": criteria_names[p],
                "C": criteria_names[c],
                "Delta(P)": float(contributions[p]),
                "Delta(C)": float(contributions[c]),
                "Delta(P)+Delta(C)": s,
                "valid_tradeoff": (s > 0)
            })
    df = pd.DataFrame(rows)
    # Show valid ones first, then larger margins first

    return df.sort_values(["valid_tradeoff", "Delta(P)+Delta(C)"], ascending=[False, False]).reset_index(drop=True)



## Fonction principale : Recherche d'explication (1-1)

In [14]:
def find_explanation_1_1(x_scores, y_scores, weights, criteria_names=None, verbose=True):
    """
    Trouve une explication de type (1-1) pour x > y en utilisant un programme lineaire
    
    Args:
        x_scores: array des notes de x
        y_scores: array des notes de y
        weights: array des poids
        criteria_names: liste des noms des criteres (optionnel)
        verbose: afficher les details
    
    Returns:
        explanation: liste de tuples (P, C) ou None si pas d'explication
        status: statut de la resolution
    """
    n_criteria = len(x_scores)
    
    if criteria_names is None:
        criteria_names = [f"C{i}" for i in range(n_criteria)]
    
    # Calculer les contributions
    contributions = compute_contributions(x_scores, y_scores, weights)
    
    if verbose:
        print("\n=== Analyse de la comparaison x > y ===")
        print("\nContributions par critere:")
        for i, name in enumerate(criteria_names):
            print(f"  {name}: {contributions[i]:+.1f}")
    
    # Classifier les criteres
    pros, cons, neutral = classify_criteria(contributions)
    
    if verbose:
        print(f"\npros(x,y) = {{{', '.join([criteria_names[i] for i in pros])}}}")
        print(f"cons(x,y) = {{{', '.join([criteria_names[i] for i in cons])}}}")
        print(f"neutral(x,y) = {{{', '.join([criteria_names[i] for i in neutral])}}}")
    
    # Verifier que x > y
    total_contribution = np.sum(contributions)
    if total_contribution <= 0:
        print(f"\nErreur: x n'est pas meilleur que y (contribution totale = {total_contribution:.2f})")
        return None, "INVALID"
    
    # Cas trivial : pas de cons
    if len(cons) == 0:
        if verbose:
            print("\nAucun critere defavorable a x. Pas besoin d'explication.")
        return [], "TRIVIAL"
    
    # Creer le modele Gurobi
    model = gp.Model("explanation_1_1")
    model.Params.OutputFlag = 0 if not verbose else 1
    
    # Variables de decision : z[p][c] = 1 si on utilise le trade-off (p, c)
    # On ne cree des variables QUE pour les paires valides (contribution positive)
    epsilon = 1e-9
    z = {}
    valid_pairs = []
    
    for p in pros:
        for c in cons:
            contrib_sum = float(contributions[p] + contributions[c])
            if contrib_sum > epsilon:  # Seulement les paires valides
                z[p, c] = model.addVar(vtype=GRB.BINARY, name=f"z_{criteria_names[p]}_{criteria_names[c]}")
                valid_pairs.append((p, c))
    
    if verbose:
        print(f"\nPaires valides (contribution > 0):")
        for p, c in valid_pairs:
            contrib = float(contributions[p] + contributions[c])
            print(f"  ({criteria_names[p]}, {criteria_names[c]}): {contributions[p]:+.1f} + ({contributions[c]:+.1f}) = {contrib:+.1f}")
    
    # Verifier que chaque element de cons peut etre couvert
# Certificate (simple): if a con criterion has NO valid partner, a (1-1) explanation is impossible.
    uncoverable = []
    for c in cons:
        if not any((p, c) in z for p in pros):
            uncoverable.append(c)

    if len(uncoverable) > 0:
        if verbose:
            print("\n=== No (1-1) explanation exists (CERTIFICATE) ===")
            print("Reason: at least one 'con' criterion cannot be paired with any 'pro' criterion")
            print("under the rule Delta(P)+Delta(C) > 0.")
            print("Uncoverable cons criteria:")
            for c in uncoverable:
                print(f" - {criteria_names[c]} (Delta={contributions[c]:+.1f})")
        return None, "INFEASIBLE_UNCOVERABLE"

    
    model.update()
    
    # Contrainte 1 : Chaque element de cons doit etre couvert exactement une fois
    for c in cons:
        model.addConstr(
            gp.quicksum(z[p, c] for p in pros if (p, c) in z) == 1,
            name=f"cover_cons_{criteria_names[c]}"
        )
    
    # Contrainte 2 : Chaque element de pros peut etre utilise au plus une fois
    for p in pros:
        model.addConstr(
            gp.quicksum(z[p, c] for c in cons if (p, c) in z) <= 1,
            name=f"use_pros_{criteria_names[p]}"
        )
    
    # Objectif : minimiser le nombre de trade-offs (longueur de l'explication)
    model.setObjective(
        gp.quicksum(z[p, c] for p in pros for c in cons if (p, c) in z),
        GRB.MINIMIZE
    )
    
    # Resoudre
    model.optimize()
    
    # Analyser le resultat
    if model.status == GRB.OPTIMAL:
        explanation = []
        if verbose:
            print("\n=== Explication de type (1-1) trouvee ===")
            print(f"Longueur de l'explication: {int(model.objVal)}")
            print("\nTrade-offs:")
        
        for p in pros:
            for c in cons:
                if (p, c) in z and z[p, c].X > 0.5:
                    explanation.append((p, c))
                    if verbose:
                        contrib_p = contributions[p]
                        contrib_c = contributions[c]
                        contrib_sum = contrib_p + contrib_c
                        print(f"  ({criteria_names[p]}, {criteria_names[c]}): "
                              f"{contrib_p:+.1f} + ({contrib_c:+.1f}) = {contrib_sum:+.1f} > 0")
        
        return explanation, "OPTIMAL"
    
    elif model.status == GRB.INFEASIBLE:
        if verbose:
            print("\n=== No (1-1) explanation exists ===")
            print("Gurobi status: INFEASIBLE. Computing IIS (certificate)...")

        # IIS is a standard infeasibility certificate: a minimal set of constraints that cannot all hold together.
        model.computeIIS()

        if verbose:
            print("\nIIS constraints (certificate):")
            for constr in model.getConstrs():
                if constr.IISConstr:
                    print(" -", constr.ConstrName)

        return None, "INFEASIBLE_IIS"


    else:
        if verbose:
            print(f"\nStatut de resolution: {model.status}")
        return None, f"STATUS_{model.status}"


def format_explanation(explanation, contributions, criteria_names):
    """
    Formate l'explication en langage naturel
    """
    if not explanation:
        return "Aucune explication necessaire (pas de criteres defavorables)."
    
    lines = ["x est meilleur que y car:"]
    for p, c in explanation:
        lines.append(f"  - l'avantage de x sur y en {criteria_names[p]} "
                    f"pese plus fort que son desavantage en {criteria_names[c]}")
    
    return "\n".join(lines)

## Test 1 : Comparaison x > y (doit trouver une explication)

Selon le sujet, nous devrions trouver l'explication : {(A, C), (D, F), (E, G)}

In [15]:
# Donnees du probleme
criteria_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
full_names = ['Anatomie', 'Biologie', 'Chirurgie', 'Diagnostic', 
              'Epidemiologie', 'Forensic Pathology', 'Genetique']
weights = np.array([8, 7, 7, 6, 6, 5, 6])

x = np.array([85, 81, 71, 69, 75, 81, 88])
y = np.array([81, 81, 75, 63, 67, 88, 95])

# Afficher les donnees sous forme de tableau
df = pd.DataFrame({
    'Critere': criteria_names,
    'Nom complet': full_names,
    'Poids': weights,
    'Xavier (x)': x,
    'Yvonne (y)': y
})


print("\n" + "="*70)
print("TEST 1: Comparaison Xavier > Yvonne")
print("="*70)
print("\nDonnees:")
print(df.to_string(index=False))


TEST 1: Comparaison Xavier > Yvonne

Donnees:
Critere        Nom complet  Poids  Xavier (x)  Yvonne (y)
      A           Anatomie      8          85          81
      B           Biologie      7          81          81
      C          Chirurgie      7          71          75
      D         Diagnostic      6          69          63
      E      Epidemiologie      6          75          67
      F Forensic Pathology      5          81          88
      G          Genetique      6          88          95


In [16]:
# Trouver l'explication
explanation, status = find_explanation_1_1(x, y, weights, criteria_names)


=== Analyse de la comparaison x > y ===

Contributions par critere:
  A: +32.0
  B: +0.0
  C: -28.0
  D: +36.0
  E: +48.0
  F: -35.0
  G: -42.0

pros(x,y) = {A, D, E}
cons(x,y) = {C, F, G}
neutral(x,y) = {B}
Set parameter OutputFlag to value 1

Paires valides (contribution > 0):
  (A, C): +32.0 + (-28.0) = +4.0
  (D, C): +36.0 + (-28.0) = +8.0
  (D, F): +36.0 + (-35.0) = +1.0
  (E, C): +48.0 + (-28.0) = +20.0
  (E, F): +48.0 + (-35.0) = +13.0
  (E, G): +48.0 + (-42.0) = +6.0
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (win64 - Windows 11+.0 (26200.2))

CPU model: Intel(R) Core(TM) Ultra 9 285H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6 rows, 6 columns and 12 nonzeros (Min)
Model fingerprint: 0x0c593c2c
Model has 6 linear objective coefficients
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  B

In [17]:
# Afficher l'explication en langage naturel
if explanation is not None:
    contributions = compute_contributions(x, y, weights)
    print("\n" + format_explanation(explanation, contributions, criteria_names))


x est meilleur que y car:
  - l'avantage de x sur y en A pese plus fort que son desavantage en C
  - l'avantage de x sur y en D pese plus fort que son desavantage en F
  - l'avantage de x sur y en E pese plus fort que son desavantage en G
